# Milestone 2: Democracy does cause growth

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import altair as alt    
#For the choropleth map
import geopandas as gpd
import matplotlib.pyplot as plt

## 1. Obtaining the data

In [ ]:
DATA_DIR = Path("C:/Users/josep/Desktop/DataViz/Data_Viz_Static/data")
dir=DATA_DIR/'iso.csv'
iso=pd.read_csv(dir)

In [ ]:
dir=DATA_DIR/'API_NY.GDP.PCAP.PP.KD_DS2_en_csv_v2_22981.csv'
GDP=pd.read_csv(dir, skiprows=4)
GDP=GDP.drop(columns=['Unnamed: 69'])
GDP_long=GDP.melt(id_vars=['Country Name','Country Code','Indicator Name','Indicator Code'],var_name='Year',value_name='GDP')
GDP_long['Year']=GDP_long['Year'].astype(int)  
GDP=GDP_long[['Country Name', 'Country Code', 'Year', 'GDP']] 
GDP.rename(columns={'GDP':'GDP_per_capita'}, inplace=True)

GDP=pd.merge(GDP, iso, left_on='Country Code', right_on='iso3', how='inner')

In [ ]:
dir=DATA_DIR/'All_data_FIW_2013-2025.xlsx'
Freedom_House=pd.read_excel(dir, sheet_name="FIW13-25", skiprows=1)
Freedom_House=Freedom_House[['Country/Territory', 'Region', 'Edition',  'Status', 'Total']]

Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Bahamas', 'Bahamas, The')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Cabo Verde', 'Cape Verde')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Congo (Brazzaville)', 'Republic of the Congo')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Congo (Kinshasa)', 'Democratic Republic of the Congo')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('North Macedonia', 'Macedonia')
Freedom_House['Country/Territory']=Freedom_House['Country/Territory'].str.replace('Slovakia', 'Slovak Republic')

Freedom_House=pd.merge(Freedom_House, iso, left_on='Country/Territory', right_on='country', how='inner')


In [ ]:
dir_p5=DATA_DIR/'p5v2018.xls'
polityv=pd.read_excel(dir_p5,engine='xlrd')
polityv=polityv[['scode', 'country','year','polity2', 'polity', 'durable']]
polityv["year_polity"]=polityv["year"]

polityv['country']=polityv['country'].str.replace('Bosnia', 'Bosnia and Herzegovina')
polityv['country']=polityv['country'].str.replace('Congo Brazzaville', 'Republic of the Congo' )
polityv['country']=polityv['country'].str.replace('Congo-Brazzaville', 'Republic of the Congo' )
polityv['country']=polityv['country'].str.replace('Congo Kinshasa', 'Democratic Republic of the Congo' )
polityv['country']=polityv['country'].str.replace("Cote D'Ivoire", "Cote d'Ivoire" )
polityv['country']=polityv['country'].str.replace("Gambia", "Gambia, The" )
polityv['country']=polityv['country'].str.replace("Ivory Coast", "Cote d'Ivoire")
polityv['country']=polityv['country'].str.replace("Korea North", "North Korea")
polityv['country']=polityv['country'].str.replace("Korea South", "South Korea")
polityv['country']=polityv['country'].str.replace("Myanmar (Burma)", "Myanmar")
polityv['country']=polityv['country'].str.replace("USSR", "Soviet Union")
polityv['country']=polityv['country'].str.replace("Sudan-North", "Sudan")

polityv=pd.merge(polityv, iso, left_on='country', right_on='country', how='inner')

## 2. Functions

In [ ]:

#Create a scatter plot function
def create_scatter_plot(data, x_col, y_col, title_label, x_label, y_label, tooltip_var, width_n, height_n, foot_text, color_type, color_R):
    chart=alt.Chart(data).mark_circle(size=60, color=color_type).encode(
    x=alt.X(x_col, title=x_label),
    y=alt.Y(y_col, title=y_label),
    tooltip=[tooltip_var, x_col, y_col]
    
    #IA:  how to add footnote to a altair graphic? (Didn't find it in documentation)
    #In other graphics I opted for using concat as I added the title here
    ).properties(width=width_n, height=height_n,
                 title=alt.TitleParams(
            [foot_text],
            baseline='bottom',
            orient='bottom',
            anchor='start',
            fontWeight='normal',
            fontSize=10,
            dy=20, dx=20
        ))

    
    chart_=chart + chart.transform_regression(x_col, y_col).mark_line(color=color_R)

    #IA:  how to add additional text to altair graphic
    title = (
    alt.Chart({"values": [{"text": f'{title_label}'}]})
      .mark_text(size=15, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=width_n, height=28))
    
    chart_final= alt.vconcat(title, chart_, spacing=4)
    
    return chart_final
    

## 3. Charts

### Graphic 1: Relation between GDP and Democracy

In [ ]:
merged_data=pd.merge(Freedom_House, GDP, left_on=['iso3','Edition'], right_on=['iso3','Year'], how='inner')
merged_data=merged_data[merged_data['Year']==2024]
merged_data['GDP_per_capita']=merged_data['GDP_per_capita'].round(0)
merged_data = merged_data.rename(columns={"Total": "FH_Score"})


In [ ]:
create_scatter_plot(merged_data, 
                    'FH_Score', 'GDP_per_capita', 
                    '2024: Liberty and Democracy vs GDP per capita', 
                    'Liberty and Democrac (FH Score)', 'GDP per capita, PPP (constant 2021 international $)', 
                    'iso3',
                    600, 400,
                    "The graph shows the relationship between Freedom House Score of Democracy and Liberty and GDP per capita in 2024.",
                "#2196F3", "#F32121")
    

### Graphic 2: Relationship by regions

In [ ]:
areas=['Americas', 'Middle East', 'Europe', 'Asia']

In [ ]:
americas_data=merged_data[merged_data['Region']=='Americas']
middle_east_data=merged_data[merged_data['Region']=='Middle East']  
europe_data=merged_data[merged_data['Region']=='Europe']
asia_data=merged_data[merged_data['Region']=='Asia']


### Graphic 1
americas_plot= create_scatter_plot(americas_data, 
                    'FH_Score', 'GDP_per_capita', 
                    'Americas', 
                    'Liberty and Democrac (FH Score)', 'GDP per capita, PPP (constant 2021 international $)', 
                    'iso3',
                    300, 200,
                    "",
                "#2196F3", "#2196F3")

middle_east_plot=create_scatter_plot(middle_east_data, 
                    'FH_Score', 'GDP_per_capita', 
                    'Middle East', 
                    'Liberty and Democrac (FH Score)', 'GDP per capita, PPP (constant 2021 international $)', 
                    'iso3',
                    300, 200,
                    "",
                "#F32121", "#F32121")


europe_plot=create_scatter_plot(europe_data, 
                    'FH_Score', 'GDP_per_capita', 
                    'Europe', 
                    'Liberty and Democrac (FH Score)', 'GDP per capita, PPP (constant 2021 international $)', 
                    'iso3',
                    300, 200,
                    "",
                "#0552B8", "#0552B8")

asia_plot=create_scatter_plot(asia_data, 
                    'FH_Score', 'GDP_per_capita',  
                    'Asia',
                    'Liberty and Democrac (FH Score)', 'GDP per capita, PPP (constant 2021 international $)',
                    'iso3',
                    300, 200,
                    "",
               "#910505", "#910505") 

superior= americas_plot | europe_plot
inferior= middle_east_plot | asia_plot

total_graphic=alt.vconcat(superior, inferior)


In [ ]:

title = (
    alt.Chart({"values": [{"text": "2024: Relationship between Liberty and Democracy and GDP by Regions"}]})
      .mark_text(size=20, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))

foot_note = (
    alt.Chart({"values": [{"text": "The positive relationship is stronger in the Americas and Europe, while in Asia and Middle East seems nule"}]})
      .mark_text(size=12, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))
    

final = alt.vconcat(title, total_graphic, foot_note, spacing=4).configure(background='white').configure_view(stroke=None)


final

### Graphic 3: Map, democracy in the world

In [ ]:
df=merged_data[['iso3', "FH_Score"]]
df.rename(columns={'FH_Score':'value'}, inplace=True)

In [ ]:
#IA: How to create a colorplath map in altair?
#This was particully challenging, because I wasn't able to add a layer of data to the sintaxys published in documentation of altair
#Also, responses of IA did not work, so I had to try and find a jason that matched the requirements (topo_url)
#   
def map(df, title_label, color_1, color_2, foot_note_label):
    # TopoJSON with country properties including `name`
    topo_url = "https://cdn.jsdelivr.net/npm/visionscarto-world-atlas/world/110m.json"

    countries = alt.Data(
        url=topo_url,
        format={"type": "topojson", "feature": "countries"}
    )

    base = (
        alt.Chart(countries)
        .mark_geoshape(fill="#e6e6e6", stroke="#aaaaaa", strokeWidth=0.5)
        .project("equalEarth")
        .properties(width=900, height=480, title= title_label)
    )

    # Choropleth layer
    choropleth = (
        alt.Chart(countries)
        .mark_geoshape(stroke="#000000", strokeWidth=.2)
        .transform_lookup(
            lookup="properties.a3",        # iso name in the map
            from_=alt.LookupData(df, key="iso3", fields=["value","iso3"])
        )
        .encode(
            color=alt.condition(
                "datum.value != null",                              
                alt.Color("value:Q", title="Value",
                            scale=alt.Scale(range=[color_1, color_2])),
                alt.value("#e6e6e6")                              
            ),
            tooltip=[
                alt.Tooltip("properties.name:N", title="Country"),
                alt.Tooltip("iso3:N", title="ISO3"),
                alt.Tooltip("value:Q", title="Value", format=".2f")
            ]
        )
    )

    map=(base + choropleth)
    

    foot_note = (
    alt.Chart({"values": [{"text": foot_note_label}]})
      .mark_text(size=10, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))
    
    #.configure_view(stroke=None)
    final = alt.vconcat(map, foot_note, spacing=6).configure(background='white').configure_view(stroke=None)
    return final

In [ ]:
map(df, "2024: Level of Liberty and Democracy in the World", "#DD1313", "#95f3f7", "Liberal Democracy is more consolidated in the west world, while this does not mean higher GDP")

### Graphic 4: GDP by level of democracy:

In [ ]:
### Graphic 4: GDP by level of democracy:

def boxplot_graph(data, x_col, y_col, x_axis, y_axis, title_lab, g_height, g_width, color_1):
    boxplot = alt.Chart(data).mark_boxplot(color=color_1).encode(
        x=alt.X(x_col, title=x_axis), 
        y=alt.Y(y_col,title=y_axis),
        #color=alt.Color(color_1)
    ).properties(
        title=title_lab,
        height=g_height, width=g_width)

    return boxplot

In [ ]:
merged_data["Status_Democracy"] = merged_data["Status"].map({
    "F": "1.Free",
    "PF": "2.Partly Free",
    "NF": "3.Not Free"})

In [ ]:
box=boxplot_graph(merged_data, "Status_Democracy", "GDP_per_capita",
               "Status of Country", "GDP per capita PPP 2011 USD",
                 "2024: GDP per capita by status of Country",
                 400, 600,
                 "#0ada79")



foot_note = (
    alt.Chart({"values": [{"text": "The distribution of the free nations is cleary higher than Not and Partially Nations."
    "But, there are no clear diference between these two groups"}]})
      .mark_text(size=12, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))
    

final = alt.vconcat(box, foot_note, spacing=6).configure(background='white').configure_view(stroke=None)
final


### Graphic 5: Diferent level of democracy

In [ ]:
merged_data=pd.merge(polityv, GDP, left_on=['iso3', 'year'], right_on=['iso3', 'Year'], how='inner')

In [ ]:
#we only take if year==2018
merged_data = merged_data[merged_data["year"] == 2018]


In [ ]:
box=boxplot_graph(merged_data, "polity2", "GDP_per_capita",
               "Level of Democracy (PolityV Index 1-10)", "GDP per capita PPP 2011 USD",
                 "2018: GDP per capita by status of Country",
                 400, 600,
                 "#10bcf1")



foot_note = (
    alt.Chart({"values": [{"text": "In this graphic the relationship is less clear. While in democracies there is a positve relationship. "
    "The least free countries are in mean richier than average democracies"}]})
      .mark_text(size=12, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))
    

final = alt.vconcat(box, foot_note, spacing=6).configure(background='white').configure_view(stroke=None)
final


Graphic 6. Population living in different regimen 

In [ ]:
dir=DATA_DIR/'fake_political_regime_shares_1816_2015.csv'
share_data=pd.read_csv(dir)

In [ ]:
regime_cols = ["democracy", "open_anocracy", "closed_anocracy", "authoritarian", "colony"]
#IA: Create a fake date based on this categories

chart = (
    alt.Chart(share_data)
      .transform_fold(regime_cols, as_=["regime", "value"])
      .mark_area()
      .encode(
          x=alt.X("year:Q", title="Year", axis=alt.Axis(format="d", tickCount=10)),
          y=alt.Y("sum(value):Q", stack="normalize",
                  title="Share of regime types", axis=alt.Axis(format="%")),
          color=alt.Color(
              "regime:N",
              title="Regime type",
              scale=alt.Scale(
                  domain=["democracy","open_anocracy","closed_anocracy","authoritarian","colony"],
                  range=["#1EB173", "#10bcf1", "#22428f", "#d42d1a", "#cccccc"]
              )
          ),

      )
      .properties(width=600, height=400, title="People living in different regimens (1820-2020) - Generated Data")
)

foot_note = (
    alt.Chart({"values": [{"text": "Eventhought the relationship across countries is not clear. There amount of people living in democracies  have been increased"}]})
      .mark_text(size=12, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))
    
final = alt.vconcat(chart, foot_note, spacing=6).configure(background='white').configure_view(stroke=None)


final

### Graphic 7. Growth Post Democratization

In [ ]:
#Historic GDP
dir_p5=DATA_DIR/'mpd2023_web.xlsx'
df=pd.read_excel(dir_p5, sheet_name="GDPpc")

In [ ]:
#Transforming the excel
first_col = df.columns[0]
region_row = df.loc[df[first_col].eq('Region')].iloc[0]
code_row   = df.loc[df[first_col].eq('year')].iloc[0]   

region_map = region_row.drop(first_col).to_dict()
code_map   = code_row.drop(first_col).to_dict()

data = df[~df[first_col].isin(['Region', 'year'])].copy()
data = data.rename(columns={first_col: 'year'})

long = data.melt(
    id_vars='year',
    var_name='country',
    value_name='gdp_pc_2011'
)

long['region'] = long['country'].map(region_map)
long['code']   = long['country'].map(code_map)

long['year'] = pd.to_numeric(long['year'], errors='coerce')
long['gdp_pc_2011'] = pd.to_numeric(long['gdp_pc_2011'], errors='coerce')

long = long.sort_values(['country','year']).reset_index(drop=True)

long = long.dropna(subset=['year', 'gdp_pc_2011'])

long.head()

In [ ]:
gdp_historic=long.copy()
#correcting Soviet Union
gdp_historic['code']=gdp_historic['code'].str.replace('SUN', 'SVU')

#
gdp_historic["growth_rate"] = gdp_historic.groupby('country')['gdp_pc_2011'].pct_change() * 100

In [ ]:
gdp_historic=pd.merge(gdp_historic, iso, left_on='code', right_on='iso3', how='inner')

In [ ]:
#Sablishing the democratization years:
df = polityv.sort_values(['country', 'year']).copy()

df['is_trans'] = (df['polity'] == -88)    
df['next_is_dem'] = df.groupby('country')['polity2'].shift(-1).ge(6)

events = (df.loc[df['is_trans'] & df['next_is_dem'], ['country','year']]
            .assign(dem_year=lambda x: x['year'] + 1)[['country','dem_year']])

In [ ]:
#IA: How to create lagged and lead variables?
ks = list(range(-10, 11))
win = (events.assign(key=1)
             .merge(pd.DataFrame({'k': ks, 'key': 1}), on='key')
             .assign(year=lambda x: x['dem_year'] + x['k'])
             [['country','year','k']])


df = polityv.merge(win, on=['country','year'], how='left')

for k in ks:
    df[f'event_{k}'] = (df['k'] == k).astype('Int64')  

df['event_time'] = df['k']  # not considering nan

df['year_event']=df["year"]-1

polityv_events=df.copy()

In [ ]:
matched_data=pd.merge(polityv_events, gdp_historic, left_on=['iso3', 'year_event'], right_on=['iso3', 'year'], how='inner')

In [ ]:
matched_data = matched_data.dropna(subset=['event_time', 'growth_rate'])
alt.data_transformers.disable_max_rows()

data=matched_data.copy()
#data = data[data["region"] == "Western Europe"].copy()

data_before=data[data["event_time"]<=0]
data_after=data[data['event_time']>=0]

line_before = alt.Chart(data_before).mark_line(color="#C21616").encode(
    x=alt.X('event_time:O', title='Years since Democratization'),
    y=alt.Y('mean(growth_rate):Q', title='Mean GDP growth per Capita (2011 USD)'),
).properties(width=600, height=400,
    title="GDP growth before and after Democratization"
)

line_after = alt.Chart(data_after).mark_line(color="#0FB45A").encode(
    x=alt.X('event_time:O', title='Years since Democratization'),
    y=alt.Y('mean(growth_rate):Q', title='Mean GDP growth per Capita (2011 USD)'),
).properties(width=600, height=400,
    title="GDP growth before and after Democratization"
)

line=line_before + line_after


foot_note = (
    alt.Chart({"values": [{"text": "Eventhought the relationship across countries is not clear. There is a clear improvement (in mean) "
                           "in the countries that had transisionated to democracy"}]})
      .mark_text(size=10, baseline="top", align="center")
      .encode(
          text="text:N",
          y=alt.value(0)
      )
      .properties(width=600, height=28))
    
final = alt.vconcat(line, foot_note, spacing=6).configure(background='white').configure_view(stroke=None)
final

### Graphic 8, 9  and 10 .Waves of democratization, independence, and authoritarian reversals

In [ ]:
swift = df[df["polity"].isin([-88, -77])].copy()
swifts=swift.copy()
swifts=swifts.rename(columns={'iso3_x': 'iso3'})

In [ ]:
african_wave=swifts[swifts['year'].between(1990,1995)]
african_wave=african_wave.groupby(['iso3'], as_index=False).agg(
    value=('country', 'count'))


#african_wave['value']=1

#african_wave=african_wave[['iso3', 'value']]


map(african_wave, "1990–1995: Democratization and regime changes in Africa (number of regime switches)", "#91b5eb", "#1900f7", 
                 "In a short span, many African countries liberalized politics and shifted from single-party or military rule to multiparty systems through regime change and elections")


In [ ]:
arab_spring=swifts[swifts['year'].between(2010,2012)]
arab_spring=arab_spring.groupby(['iso3'], as_index=False).agg(
    value=('country', 'count'))

map(arab_spring, "2010-2012: Arab Spring (number of regime switches) ","#f37474", "#b30404",
                 "In a short span, protests took place in several Arab countries, achieving (in some cases) elections or regime collapse")


In [ ]:
between_wars=swifts[swifts['year'].between(1922,1938)]
between_wars=between_wars.groupby(['iso3'], as_index=False).agg(
    value=('country', 'count'))

map(between_wars, "1922–1938: Interwar period and consolidation of authoritarism (number of regimen switches) ","#f37474", "#b30404", 
    "Between 1922 and 1938, political instability drove many countries in Europe, the Middle East, and Latin America to switch toward authoritarian regimenes")
